# Use retail_db
### Problem Statement 
#####          Get Daily revenue by product considering completed and closed orders
#####          Data need to be sorted by ascending order by date and then descending order by revenue completed for each product for each day
#####          Data should be delimeted by "," in this order [order_date , daily revenue_per_product , product_name ]

### Final output need to be stored in avro & text format



In [2]:
orders = sc.textFile("retail_db/orders")
orderItems = sc.textFile("retail_db/order_items")

for i in orders.take(1):print i

for i in orderItems.take(1):print i    
    
#print orders.map(lambda o:o.split(",")[3]).distinct().collect()    

ordersFiltered = orders.filter(lambda o: o.split(",")[3] in ["COMPLETE","CLOSED"] )

ordersMap = ordersFiltered.map(lambda o: ( int(o.split(",")[0]) , o.split(",")[1]) )
orderItemsMap = orderItems.map(lambda oi: ( int(oi.split(",")[1]), ( int(oi.split(",")[2]) ,float(oi.split(",")[4]) ) ) )
                               
ordersOrderItemsJoin = ordersMap.join(orderItemsMap)
#print ordersOrderItemsJoin.take(1)
#[(65536, (u'2014-05-16 00:00:00.0', (957, 299.98)))]

orderJoinMap = ordersOrderItemsJoin.map(lambda k: ( (k[1][0],k[1][1][0]) , k[1][1][1] ) )
#print orderJoinMap.take(1)
#[((u'2014-05-16 00:00:00.0', 957), 299.98)]

revenuePerProduct = orderJoinMap.reduceByKey(lambda x,y:x+y)
#print revenuePerProduct.take(5)

orderProductMap = revenuePerProduct.map(lambda r: (r[0][1], r))

products = open("/home/zubairidrees/hadoop-2.7.3/data/retail_db/products/part-00000").read().splitlines()
productRDD = sc.parallelize(products)
productMap = productRDD.map(lambda p: (int(p.split(",")[0]),p.split(",")[2]  ))

ordersProductJoin = orderProductMap.join(productMap)

#print ordersProductJoin.take(1)
#[ (24,  ( ( (u'2013-12-04 00:00:00.0', 24), 159.98), 'Elevation Training Mask 2.0'))]
productRevenueByDate = ordersProductJoin.map(lambda op: ((op[1][0][0][0], -op[1][0][1]), \
                                                         ( op[1][0][0][0]+ ","+str( op[1][0][1]) +","+ op[1][1] ) ) )

#print productRevenueByDate.take(1)

print "Sorted Product"
sortedProducts = productRevenueByDate.sortByKey().map(lambda s: s[1])
#for i in sortedProducts.take(2): print i
 
sortedProducts. \
coalesce(2).saveAsTextFile("/user/zubairdrees/daily_revenue_by_products")

#productRevenueByDate = ordersProductJoin.map(lambda op: ((op[1][0][0][0], -op[1][0][1]), \
#                                                         ( op[1][0][0][0],str( op[1][0][1]) , op[1][1] ) ) )

#sortedProducts = productRevenueByDate.sortByKey().map(lambda s: s[1])

#productsDF = sortedProducts.toDF( schema = ["date", "revenue", "product_name"] )  
#print sc.textFile("/user/zubairdrees/daily_revenue_by_product").take(5)

#help(productsDF.write)
#productsDF.save("/user/zubairidrees/revenue_per_order_id_json","com.databricks.spark.avro")

#productsDF.write("/user/zubairidrees/revenue_per_order_id_avro","com.databricks.spark.avro")   



1,2013-07-25 00:00:00.0,11599,CLOSED
1,1,957,1,299.98,299.98
Sorted Product


Py4JJavaError: An error occurred while calling o368.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://localhost:9000/user/zubairdrees/daily_revenue_by_products already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1184)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1161)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1161)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:358)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1161)
	at org.apache.spark.rdd.PairRDDFunctions$$anonf